In [ ]:
!apt-get install -y build-essential libboost-all-dev
!pip install https://github.com/kpu/kenlm/archive/master.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     \ 553.6 kB 3.7 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp311-cp311-linux_x86_64.whl size=3185076 sha256=db3218a88d77d4022ad31aa69821468908cc908e8e7e21136d884fcd8ca98df5
  Stored in directory: /tmp/pip-ephem-wheel-cache-s7jjbvsx/wheels/4e/ca/6a/e5da175b1396483f6f410cdb4cfe8bc8fa5e12088e91d60413
Successfully built kenlm


In [ ]:
!pip install jiwer editdistance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load your labels.csv file from line-based data
csv_path = '/content/labels.csv'
df = pd.read_csv(csv_path)

# Filter only line-based training data
line_df = df[(df['type'] == 'text-line') & (df['split'] == 'train')]

# Extract labels (text lines)
lines = line_df['label'].dropna().astype(str).tolist()

# Save to train_text.txt
with open('train_text.txt', 'w', encoding='utf-8') as f:
    for line in lines:
        f.write(line.strip() + '\n')

print(f" Saved {len(lines)} lines to train_text.txt")

✅ Saved 32212 lines to train_text.txt


In [ ]:
!apt-get update
!apt-get install -y build-essential libboost-all-dev cmake unzip
!wget https://github.com/kpu/kenlm/archive/master.zip -O kenlm.zip
!unzip kenlm.zip
%cd kenlm-master
!mkdir -p build && cd build && cmake .. && make -j4


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,561 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,050 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Pac

In [ ]:
# Train Unigram
!./build/bin/lmplz -o 1 < /content/train_text.txt > /content/amharic_unigram.arpa

# Train Bigram
!./build/bin/lmplz -o 2 < /content/train_text.txt > /content/amharic_bigram.arpa


# Train Trigram
!./build/bin/lmplz -o 3 < /content/train_text.txt > /content/amharic_trigram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/train_text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 117406 types 32529
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:390348
Statistics:
1 32529 D1=0.686588 D2=1.23098 D3+=1.41702
Memory estimate for binary LM:
type      kB
probing 1397 assuming -p 1.5
probing 1524 assuming -r models -p 1.5
trie     945 without quantization
trie     852 assuming -q 8 -b 8 quantization 
trie     945 assuming -a 22 array pointer compression
trie     852 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:390348
=== 4/5 Calculating and writing order-interpolated probabilities ===
Chain sizes: 1:390348
=== 5/5 Writing ARPA model ===
----5---10---15---20---25---30---

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load your character‐level labels
df = pd.read_csv('/content/char_labels.csv')
df_char = df[df.type=='character']

# Fit a LabelEncoder on the training split only
le = LabelEncoder()
le.fit(df_char[df_char.split=='train']['label'])

# Your list of characters, in index order 0…num_classes-1
char_list = list(le.classes_)

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import kenlm
import pandas as pd
from jiwer import wer
from editdistance import eval as edit_distance

# ─── 1. Load the trained character CNN ───
model = tf.keras.models.load_model('/content/amharic_char_cnn.h5')

# ─── 2. Prepare test metadata ───
# CSV format: filename,label
test_df = pd.read_csv('/content/test_lines.csv')
# e.g. test_lines.csv:
# line001.png,ሰላም ለዓለም
# line002.png,አማርኛ ቋንቋ

IMAGE_DIR = '/content/test_line_images'  # folder containing your 48×128 PNGs

# ─── 3. Load your KenLM models ───
bigram_lm = kenlm.Model('/content/amharic_bigram.arpa')
trigram_lm = kenlm.Model('/content/amharic_trigram.arpa')

# ─── 4. Build index→char list ───
# We need the exact mapping you used during character training:
# If you saved `le.classes_`, reload it here:
# char_list = list(np.load('char_classes.npy'))
# char_classes.npy was saved via: np.save('char_classes.npy', le.classes_)

blank_token = ''  # or a special char you reserved

# After you load model, char_list, test_df, LM models…

def debug_one(fn, gt):
    img = cv2.imread(os.path.join(IMAGE_DIR, fn), cv2.IMREAD_GRAYSCALE)
    raw = sliding_window_argmax(img, model)
    ctc = ctc_collapse(raw)
    bi  = lm_rescore(ctc, bigram_lm)
    tri = lm_rescore(ctc, trigram_lm)
    print(f"GT  : {gt}")
    print(f"RAW : {raw}")
    print(f"CTC : {ctc}")
    print(f"BI  : {bi}")
    print(f"TRI : {tri}")
    print("-"*40)

# Print first 5
for i, row in test_df.head(5).iterrows():
    debug_one(row.filename, row.label)


# ─── 5. Helper functions ───

def sliding_window_argmax(img, model, window_w=32, stride=16):
    """Stage 1: Raw sliding-window + argmax"""
    preds = []
    h, w = img.shape
    for x in range(0, w-window_w+1, stride):
        patch = img[:, x:x+window_w]
        patch = cv2.resize(patch, (32,32)).astype('float32')/255.0
        patch = patch[..., None][None,...]   # (1,32,32,1)
        prob  = model.predict(patch, verbose=0)[0]  # (num_classes,)
        preds.append(char_list[np.argmax(prob)])
    return ''.join(preds)

def ctc_collapse(seq, blank=blank_token):
    """Stage 2: merge repeats & drop blanks"""
    out, prev = [], None
    for ch in seq:
        if ch == prev or ch == blank:
            prev = ch
            continue
        out.append(ch)
        prev = ch
    return ''.join(out)

def lm_rescore(seq, lm_model, alpha=1.0):
    """
    Stage 3/4: naive LM rescoring of one candidate string.
    For a full beam search you’d generate N-best, but here
    we’ll just return the raw sequence scored by the LM.
    """
    s = seq.replace('  ',' ').strip()
    # kenlm score is log10 probability; higher is better
    score = lm_model.score(s, bos=True, eos=True)
    return s  # placeholder: for real rescoring you'd pick highest-scoring variant

def cer(ref, hyp):
    """Character Error Rate"""
    return edit_distance(ref, hyp) / max(len(ref),1)

# ─── 6. Evaluation Loop ───

stages = ['raw', 'ctc', 'uni', 'tri']
metrics = {st: {'cer':[], 'wer':[]} for st in stages}

for _, row in test_df.iterrows():
    fn, gt = row.filename, row.label
    img = cv2.imread(os.path.join(IMAGE_DIR, fn), cv2.IMREAD_GRAYSCALE)

    # 1) RAW
    raw_pred = sliding_window_argmax(img, model)
    metrics['raw']['cer'].append(cer(gt, raw_pred))
    metrics['raw']['wer'].append(wer(gt, raw_pred))

    # 2) CTC
    ctc_pred = ctc_collapse(raw_pred)
    metrics['ctc']['cer'].append(cer(gt, ctc_pred))
    metrics['ctc']['wer'].append(wer(gt, ctc_pred))

    # 3) Unigram LM
    uni_pred = lm_rescore(ctc_pred, bigram_lm)
    metrics['uni']['cer'].append(cer(gt, uni_pred))
    metrics['uni']['wer'].append(wer(gt, uni_pred))

    # 4) Trigram LM
    tri_pred = lm_rescore(ctc_pred, trigram_lm)
    metrics['tri']['cer'].append(cer(gt, tri_pred))
    metrics['tri']['wer'].append(wer(gt, tri_pred))

# ─── 7. Print Summary ───
for st in stages:
    avg_cer = np.mean(metrics[st]['cer'])*100
    avg_wer = np.mean(metrics[st]['wer'])*100
    print(f"{st:>3}  CER: {avg_cer:5.2f}%   WER: {avg_wer:5.2f}%")


GT  : ቀጠሉ፡፡ ጉልላት እሮጦ ሄዶ 
RAW : ጬጩሡጩሡጬጬ
CTC : ጬጩሡጩሡጬ
BI  : ጬጩሡጩሡጬ
TRI : ጬጩሡጩሡጬ
----------------------------------------
GT  : ሳይሸራረፉ የሚከበሩበት አግባብ 
RAW : ጬጬጬጬጩጬጬ
CTC : ጬጩጬ
BI  : ጬጩጬ
TRI : ጬጩጬ
----------------------------------------
GT  : ለኮምፒዩቲንግ ፋኩልቲ ባህርዳር 
RAW : ጬጬጬጬጩጬጩ
CTC : ጬጩጬጩ
BI  : ጬጩጬጩ
TRI : ጬጩጬጩ
----------------------------------------
GT  : ሁፍ የሰፈሩት ሃሳቦች፤ በተዘዋዋሪ 
RAW : ጬጬጬጬጬጬጬ
CTC : ጬ
BI  : ጬ
TRI : ጬ
----------------------------------------
GT  : ጉልላት ከተመረቀ በኋላ እግሩ 
RAW : ጬጬጬጬጩጬጬ
CTC : ጬጩጬ
BI  : ጬጩጬ
TRI : ጬጩጬ
----------------------------------------
raw  CER: 100.40%   WER: 100.00%
ctc  CER: 100.40%   WER: 100.00%
uni  CER: 100.40%   WER: 100.00%
tri  CER: 100.40%   WER: 100.00%
